In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
all_stim = pd.read_csv("AllStimuli.csv")

print(all_stim)

     Procedure  ProcedureID    target1    target2     target3     target4  \
0      VisProc            1     bright      crowd        bolt        bolt   
1      VisProc            1        bed      grass        tape        view   
2      VisProc            1        mug      frame        hill        hill   
3      VisProc            1       clay       flea        lawn        lawn   
4      VisProc            1       shop       flat        chip        chip   
..         ...          ...        ...        ...         ...         ...   
91  FalAudProc            4   full_fal   draw_fal    fort_fal    vote_fal   
92  FalAudProc            4    art_fal    bit_fal  spring_fal  spring_fal   
93  FalAudProc            4   love_fal   jazz_fal   truck_fal   truck_fal   
94  FalAudProc            4   slow_fal   land_fal    bank_fal    girl_fal   
95  FalAudProc            4  staff_fal  march_fal    ship_fal    ship_fal   

    Expected_Response  
0                   1  
1                   0  
2  

Current 17 trials of each type per run. Runs would be 34 trials of each type
For 3 runs, 12 trials of each type would give more total trials (maybe do 13 to get more signal?)
Convert main stimlus links to CSV files
Do all the randomization in this notebook, but set an initial randomization seed so this script will always generate the same stimulus files
Output is CSV and then I won't have to worry about reading or writing excel

In [3]:
# Set up the parameters that will define the types of runs to generate
NumRuns = 9
NumTrialsPerType= 12
NumTrials = 4 * NumTrialsPerType

# The number of trials per type that should have a reponse (i.e. repeated stimulus)
NumResponsePerType = 2
NumNonresponsePerType = NumTrialsPerType-NumResponsePerType

In [4]:
# Create the interstimulus intervals for all trials.
# Event related design with 8-16sec ITIs with deceasing frequency
# Doing one more of the 2nd shortest and one less of the shortest
# trial to give slightly more spacing to trials
ITIs = np.concatenate((np.tile(8,int(np.floor(NumTrials/2))-1), 
        np.tile(10,int(np.floor(NumTrials/4))+1), 
        np.tile(12,int(np.floor(NumTrials/8))),
        np.tile(14,int(np.ceil(NumTrials/16))),
        np.tile(16,int(np.ceil(NumTrials/32)))))
# The end result should be one fewer ITI than the number of trials
# since the final trial will have a longer null period before
# the end of the run
if len(ITIs) > (NumTrials-1):
    print("ERROR: Messed up trial counts")
elif len(ITIs) < (NumTrials-1):
    AddTrials = NumTrials - 1 - len(ITIs)
    ITIs.append(np.tile(12, AddTrials))
print(ITIs)
print(len(ITIs))

[ 8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8 10
 10 10 10 10 10 10 10 10 10 10 10 10 12 12 12 12 12 12 14 14 14 16 16]
47


In [5]:
# Create
# Note there are 24 trials per type so that number is 
random_state = 525600
random_state_increment = 42
for runidx in range(9): #NumRuns):
    tmprun = pd.DataFrame(columns = all_stim.columns)
    for ProcedureID in range(4):
        # Select NumResponsePerType trials with responses
        tmprun = pd.concat([tmprun,
            all_stim[(all_stim['ProcedureID']==(ProcedureID+1)) & (all_stim['Expected_Response']==1)].sample(n=NumResponsePerType, random_state=random_state)])
        random_state+=random_state_increment
        # Select NumNonresponsePerType trials without responses
        tmprun = pd.concat([tmprun,
            all_stim[(all_stim['ProcedureID']==(ProcedureID+1)) & (all_stim['Expected_Response']==0)].sample(n=NumNonresponsePerType, random_state=random_state)])
        random_state+=random_state_increment
    # Create the new run and initialize the first row as a null trial with 14sec of no stimulation
    # Note: This is particularly long because the headphones benefit from a long window of noise before
    #  noise cancellation starts to work
    tmp_shuffledITIs = ITIs
    randstate = np.random.RandomState(random_state)
    random_state+=random_state_increment
    randstate.shuffle(tmp_shuffledITIs)
    NullTrial = pd.DataFrame(data = {'Run': [round(runidx+1)], 'TrialN': [round(0)],'ProcedureID': [int(0)], 'Procedure': ['Null'], 'Font': ['Arial'],
                                           'ITISec': [14], 'target1': [' '],'target2': [' '],'target3': [' '],'target4': [' '],
                                           'Expected_Response': [int(0)]})
    # GeneratedRun.astype({'Run': 'int32'}).dtypes
    GeneratedRun = pd.concat([NullTrial, tmprun.sample(n=NumTrials, random_state=random_state)])
    del tmprun
    random_state+=random_state_increment
    for tidx in range(len(GeneratedRun)):
        # Run is column index 0
        GeneratedRun.iloc[tidx, 0] = round(runidx+1)
        # TrialN is column index 1
        GeneratedRun.iloc[tidx, 1] = round(tidx)
        # Procedure is column index 3
        if GeneratedRun.iloc[tidx, 3]  == 'FalVisProc':
            # Font is column index 4
            GeneratedRun.iloc[tidx,4] = 'Wingdings'
        else:
            GeneratedRun.iloc[tidx,4] = 'Arial'
        if (tidx>=1) & (tidx<(len(GeneratedRun)-1)):
            # ITISec is column index 5
            GeneratedRun.iloc[tidx, 5] = tmp_shuffledITIs[tidx-1]
        elif tidx == (len(GeneratedRun)-1):
            GeneratedRun.iloc[tidx, 5] = 18
    NullTrial['ITISec'] = 0
    NullTrial['TrialN'] = NumTrials+1
    GeneratedRun = pd.concat([GeneratedRun, NullTrial])
    GeneratedRun.to_csv(f"WordNonword_Run{runidx+1}.csv")
    del GeneratedRun
